In [1]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import pickle
import os.path
import base64
import email
from bs4 import BeautifulSoup
from datetime import datetime, timedelta, timezone

In [2]:
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

In [4]:
def getEmails():
    # Variable creds will store the user access token.
    # If no valid token found, we will create one.
    creds = None

    # The file token.pickle contains the user access token.
    # Check if it exists
    if os.path.exists('token.pickle'):

        # Read the token from the file and store it in the variable creds
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)

    # If credentials are not available or are invalid, ask the user to log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)

        # Save the access token in token.pickle file for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    # Connect to the Gmail API
    service = build('gmail', 'v1', credentials=creds)

    # Request a list of all the messages
    # Calculate the timestamp for 24 hours ago
    now = datetime.now(timezone.utc)
    yesterday = now - timedelta(days=1)
    yesterday_timestamp = int(yesterday.timestamp())
    query = f'after:{yesterday_timestamp} (category:primary OR category:updates)'
    result = service.users().messages().list(userId='me', q=query).execute()
    messages = result.get('messages')

    # Check if messages are returned
    if not messages:
        print("No new messages found.")
        return

    # Messages is a list of dictionaries where each dictionary contains a message id.
    # Iterate through all the messages
    for msg in messages:
        # Get the message from its id
        txt = service.users().messages().get(userId='me', id=msg['id']).execute()
        # Use try-except to avoid any Errors
        try:
            # Get value of 'payload' from dictionary 'txt'
            payload = txt['payload']
            headers = payload['headers']

            # Look for Subject and Sender Email in the headers
            subject = sender = None
            for d in headers:
                if d['name'] == 'Subject':
                    subject = d['value']
                elif d['name'] == 'From':
                    sender = d['value']

            # The Body of the message is in Encrypted format. So, we have to decode it.
            # Get the data and decode it with base64 decoder.

            if 'parts' in txt['payload']:
                for part in txt['payload']['parts']:
                    if part['mimeType'] == 'text/html':
                        data = part['body']['data']
                        data = data.replace("-", "+").replace("_", "/")
                        decoded_data = base64.b64decode(data)
                        soup = BeautifulSoup(decoded_data, "html.parser")

                        # Extract links and append to the text
                        for a in soup.find_all('a', href=True):
                            a_text = f"{a.text} ({a['href']})"
                            a.replace_with(a_text)

                        body = soup.get_text()

                    elif part['mimeType'] == 'text/plain':
                        data = part['body']['data']
                        data = data.replace("-", "+").replace("_", "/")
                        decoded_data = base64.b64decode(data)
                        body = decoded_data.decode('utf-8')
            else:
                data = payload['body'].get('data')
                if data:
                    data = data.replace("-", "+").replace("_", "/")
                    decoded_data = base64.b64decode(data)
                    body = decoded_data.decode('utf-8')

            # Generate a filename
            filename = f"emails/{sender}_{msg['id']}.txt"

            # Write email data to the file
            with open(filename, 'w', encoding='utf-8') as f:
                f.write(f"Subject: {subject}\n")
                f.write(f"From: {sender}\n")
                f.write("Message:\n")
                f.write(body)

        except Exception as e:
            print(f"An error occurred: {e}")
            pass

In [5]:
getEmails()

In [30]:
import os
import openai
from dotenv import load_dotenv

load_dotenv()
pinecone_api_key = os.getenv("PINECONE_API_KEY")
openai.api_key = os.getenv("OPENAI_API_KEY")
twilio_account_sid = os.getenv("TWILIO_ACCOUNT_SID")
twilio_auth_token = os.getenv("TWILIO_AUTH_TOKEN")
twilio_from = os.getenv("TWILIO_FROM")
twilio_to = os.getenv("TWILIO_TO")

In [7]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# Prompt
prompt_text = """You are an assistant tasked with summarizing and classifying emails. 

First, summarize the email content concisely and only include the links present corresponding to jobs in the email. Then, classify the email as one of the following categories based on its content. Use the criteria provided for each category:

1. **Job Applied Confirmation**: Emails confirming that a job application has been successfully received or submitted. These emails often thank the applicant for their interest in the position and provide further instructions or timelines for the next steps.

2. **Job Rejection**: Emails that inform the recipient that their application has not been successful. These emails usually contain phrases like "we regret to inform you," "we have decided to move forward with other candidates," or similar language indicating a rejection.

3. **Job Offered**: Emails that offer a job to the recipient. They often include details about the job offer, such as position title, salary, benefits, and instructions for how to accept the offer.

4. **Completed Task Corresponding to a Job**: Emails that confirm the completion of a task related to a job application or interview process, such as the submission of a coding test, assessment, or any required documents. If an email mentions that a test has been submitted or a task is completed for a job application, classify it as this.

5. **New Job Notification**: Emails that notify the recipient about new job openings or opportunities. These might come from job boards, recruiters, or companies and often include descriptions of available positions.

6. **Job Task or To Do**: Emails that include tasks or action items related to a job, such as follow-up actions after an interview, requests for additional information, or other job-related tasks that need to be completed by the recipient.

7. **Not Job Specific Email**: Emails that do not pertain to a specific job application, job offer, or job-related task. These could be newsletters, general information, or any emails not directly related to a job process.

Email: {element}

Output:
Summary: [Provide the summary here]
Classification: [Provide the classification here]
"""
prompt = ChatPromptTemplate.from_template(prompt_text)

# Summary chain
model = ChatOpenAI(temperature=0, model="gpt-4o")
summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()
summarize_chain

{
  element: RunnableLambda(lambda x: x)
}
| ChatPromptTemplate(input_variables=['element'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['element'], template='You are an assistant tasked with summarizing and classifying emails. \n\nFirst, summarize the email content concisely and only include the links present corresponding to jobs in the email. Then, classify the email as one of the following categories based on its content. Use the criteria provided for each category:\n\n1. **Job Applied Confirmation**: Emails confirming that a job application has been successfully received or submitted. These emails often thank the applicant for their interest in the position and provide further instructions or timelines for the next steps.\n\n2. **Job Rejection**: Emails that inform the recipient that their application has not been successful. These emails usually contain phrases like "we regret to inform you," "we have decided to move forward with other candidates,"

In [8]:
import classification as const

def classify_email(email_content):
    prompt_text1 = f"""You are a job email classifier. I want you to classify this email as one of the following: {', '.join(const.Classifications)}, just return the classification values. Here is the email: {email_content}. Use the criteria provided for each category:

    1. **Job Applied Confirmation**: Emails confirming that a job application has been successfully received or submitted. These emails often thank the applicant for their interest in the position and provide further instructions or timelines for the next steps.

    2. **Job Rejection**: Emails that inform the recipient that their application has not been successful. These emails usually contain phrases like "we regret to inform you," "we have decided to move forward with other candidates," or similar language indicating a rejection.

    3. **Job Offered**: Emails that offer a job to the recipient. They often include details about the job offer, such as position title, salary, benefits, and instructions for how to accept the offer.

    4. **Completed Task Corresponding to a Job**: Emails that confirm the completion of a task related to a job application or interview process, such as the submission of a coding test, assessment, or any required documents. If an email mentions that a test has been submitted or a task is completed for a job application, classify it as this.

    5. **New Job Notification**: Emails that notify the recipient about new job openings or opportunities. These might come from job boards, recruiters, or companies and often include descriptions of available positions.

    6. **Job Task or To Do**: Emails that include tasks or action items related to a job, such as follow-up actions after an interview, requests for additional information, or other job-related tasks that need to be completed by the recipient.

    7. **Not Job Specific Email**: Emails that do not pertain to a specific job application, job offer, or job-related task. These could be newsletters, general information, or any emails not directly related to a job process."""
    
    response = model(prompt_text1)
    return response.content

texts=[]
email_summaries = []
email_classifications = []
for filename in os.listdir("emails"):
		if filename.endswith(".txt"):
			filepath = os.path.join("emails", filename)
			with open(filepath, 'r', encoding='utf-8') as file:
				email_content = file.read()
				# Get the summary
				texts.append(email_content)
				input_data = {"element": email_content}
				summary = summarize_chain.invoke(input_data)
				email_summaries.append(summary)
				classification = classify_email(email_content)
				email_classifications.append(classification)

				

/Users/vinitmotwani/Desktop/EmailSummary_RAG/venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


In [9]:
email_summaries

['Summary: The email is a reminder from ID.me about using the same login for any government agency that accepts ID.me. It highlights the benefits of using an ID.me account for accessing various federal and state services, including the IRS, Social Security Administration, Veterans Affairs, and state agencies. The email includes several links to view government services and learn more about ID.me.\n\nLinks:\n1. [Link 1](https://click.marketing.id.me/?qs=b93ad48946f0212961c826c1f4638b7ad7f35012019c94ddb468cf46397ab3a17f2a0a80f9e5dd5ad2cb8ebbcc1299cb4a66dfc2c037776105f2df3ff3577bfc)\n2. [Link 2](https://click.marketing.id.me/?qs=b93ad48946f021298bd4a36a2166a0d23a5018111b6d17d1bbb38b754d6ddb74b9614b5e03d71b0b16407a08a882d9ab2cdc9f9baf38454d7fdded21c3aabc67)\n3. [Link 3](https://click.marketing.id.me/?qs=b93ad48946f02129c6cf933ac7fbbd791ed67e884b3faa9127dcae3f745bb1a5314542968d60258e45411b65b46d08d3a19d09514f764794a0a2e343952cfdae)\n4. [Link 4](https://click.marketing.id.me/?qs=b93ad48946f0

In [10]:
import uuid
import pinecone
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryStore
from langchain_pinecone import PineconeVectorStore
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain_core.vectorstores import VectorStore, VectorStoreRetriever



import os
from pinecone import Pinecone, ServerlessSpec, PodSpec

pc = Pinecone(
    api_key= pinecone_api_key
)

# Create the index and populate it
om_index = 'om-test-1'
if om_index not in pc.list_indexes().names():
    pc.create_index(
        name=om_index,
        dimension=1536,
        metric='cosine',
        spec=ServerlessSpec(
          cloud = 'aws',
          region = "us-east-1")
    )

## Uploading the documents to the vector database
vectorstore = PineconeVectorStore(
    embedding=OpenAIEmbeddings(),
    index_name=om_index
)

doc_ids = [str(uuid.uuid4()) for _ in texts]
metadata = [{"classification": x} for x in email_classifications]
summary_texts = [
    Document(page_content=s, metadata={"doc_id": doc_ids[i], "classification": email_classifications[i]})
    for i, s in enumerate(email_summaries)
]

# Add documents to the vector store
vectorstore.add_documents(documents=summary_texts)

/Users/vinitmotwani/Desktop/EmailSummary_RAG/venv/lib/python3.10/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


['bee67f7f-6616-4f76-9995-70b3a01359c0',
 '761f532a-3b07-4eba-95df-f97fabce4f3e',
 'b7ba5b4f-153a-4318-b56d-eab326ef11c7',
 'a9eb262d-b0d4-49e3-aea5-76c2e44e1122',
 'ff3db057-ccff-40b8-bb0d-066934809ca2',
 '0d800584-e34d-4ec5-8c8e-1fee789183fc',
 'ededdda1-2562-4a1d-9e80-7d485565f400',
 '3649c5e4-2677-4197-82cb-55fec27fb528',
 '6ff809bb-9942-4d09-bf6a-4cb26a6cd766',
 '0fcb2262-2d84-4ad0-8ecd-09e0e69988e3',
 '462b5acb-ecbd-4330-b12f-1b7a8866f86f',
 'da58572e-f4ab-4a32-9b1d-e13a082c8cd0',
 '6978b7f1-30e9-4f73-b474-0f0a72208e7f',
 '7625ef26-8ef7-4495-8262-54b36c04a4b9',
 'ae5e41d2-2bd8-4ed8-bac7-dfa96e82ff1d',
 '258fcf28-699d-4c2f-844a-f8ac4086cd17',
 'bf9e8413-e25f-4b34-ad3e-5de71893d4e5',
 '67efd1bc-b9db-4256-b364-5f2ba689488a',
 'dc804aa7-42dd-47fe-9a2c-29bb2dfcb5f1',
 '9bd3631b-6571-47e7-93fe-83fec1744dd7',
 'd5a1f991-5b6c-42fa-a08b-c53ecd05ce28',
 '436ba961-8e08-40af-b159-d15609472829',
 '4878d223-67bd-4e87-830c-6060a476043d',
 '4dea1c62-e1db-4036-ab98-9abbf35d1559']

In [88]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 15, "filter": {"classification": "Job Task or To Do"}})

In [11]:
def retrieve_context_for_classification(classification):
    retriever = vectorstore.as_retriever(
        search_kwargs={"k": 100, "filter": {"classification": classification}}
    )
    retrieved_docs = retriever.get_relevant_documents(classification)
    context = "\n".join([doc.page_content for doc in retrieved_docs])
    return context


from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter

# Prompt template
template = """Answer the question based only on the following context, which include email contents:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# LLM
model = ChatOpenAI(temperature=0, model="gpt-4o")

# RAG pipeline
chain = (
    {
        "context": lambda inputs: retrieve_context_for_classification(inputs["classification"]),
        "question": RunnablePassthrough()
    }
    | prompt
    | model
    | StrOutputParser()
)

In [90]:
print(chain.invoke({
    "classification": "New Job Notification",
    "question": "You are a email monitor agent, and you need to make a list of all the job roles with the company name and if same job is found in multiple emails just put it once in the list."
}))

Based on the provided context, here is a consolidated list of job roles with their respective company names, ensuring that each job is listed only once:

1. **SNOWFLAKE**
   - Software Engineer

2. **Trade Desk**
   - Data Scientist I-Value Algorithms

3. **Michaels Arts and Crafts**
   - Software Developer III
   - Software Developer

4. **Cboe**
   - Software Engineer Intern
   - Software Engineer Intern in Kansas
   - Linux Engineer
   - Senior Software Engineer - NYC
   - Senior Software Engineer (C++) - Lenexa, KS
   - Web QA Software Engineer Intern

5. **Vanguard**
   - Application Engineering Technical Lead II
   - Technical Product Owner, Workplace Services SRE

6. **Russell Tobin**
   - SQL Data Analyst
   - Data Analyst
   - Applications Developer 1 (Software Developer)

7. **Dexian - Signature Consultants**
   - Data Analyst

8. **Scale AI**
   - Machine Learning Research Engineering Intern (Summer 2025)

9. **AvidXchange, Inc.**
   - 2025 Undergraduate Summer Intern: Softw

In [26]:
classification_questions = {
    "Job Applied Confirmation": "You are an email monitor agent, and you need to make a list of all companies where a job application has been confirmed in the format [Company Name][Job Title] and if the Job Title is not mentioned put SWE role.",
    "Job Rejection": "You are an email monitor agent, and you need to make a list of all companies that have sent job rejection emails in the format [Company Name][Job Title] and if the Job Title is not mentioned put SWE role.",
    "Job Offered": "You are an email monitor agent, and you need to make a list of all companies that have a sent a job offer email in the format [Company Name][Job Title] and if the Job Title is not mentioned put SWE role.",
    "Completed Task Corresponding to a Job": "You are an email monitor agent, and you need to make a list of all completed tasks corresponding to a job application emails in the format [Company Name][Job Title] and if the Job Title is not mentioned put SWE role.",
    "New Job Notification": "You are an email monitoring agent. Your task is to compile a list of all new job notifications received. Format each entry as follows: [Company Name][Job Title], where the [Job Title] should be clickable and linked to the job URL. Ensure the [Job Title] is the clickable part that opens the corresponding URL. If the same job appears in multiple emails, include it only once.",
    "Job Task or To Do": "You are an email monitor agent, and you need to make a list of all tasks or to-dos related to job applications emails in the format [Company Name][Job Title] and if the Job Title is not mentioned put SWE role.",
    "Not Job Specific Email": "List all emails not specific to any job application."
}

In [28]:
results = {}
for classification, question in classification_questions.items():
    print(f"Processing classification: {classification}")
    context = retrieve_context_for_classification(classification)
    if context.strip():  # Check if context is not empty
        result = chain.invoke({
            "classification": classification,
            "question": question
        })
    else:
        result = "No emails associated with this classification category."
    results[classification] = result
    print(f"Result for {classification}: {result}\n")

# Output all results
for classification, result in results.items():
    print(f"Classification: {classification}\nResult: {result}\n")

Processing classification: Job Applied Confirmation
Result for Job Applied Confirmation: [IBM][Entry Level Software Developer]

Processing classification: Job Rejection
Result for Job Rejection: [Microsoft][Software Engineer]

Processing classification: Job Offered
Result for Job Offered: No emails associated with this classification category.

Processing classification: Completed Task Corresponding to a Job
Result for Completed Task Corresponding to a Job: No emails associated with this classification category.

Processing classification: New Job Notification
Result for New Job Notification: Here is the compiled list of all new job notifications received:

1. Bae Systems [[Test Design Engineer I](https://jobs.baesystems.com/global/en/job/BAE1US105022BREXTERNAL/Test-Design-Engineer-I)]
2. Wex [[Software Development Engineer 1](https://careers.wexinc.com/us/en/job/WEXWEXUSR16128EXTERNALENUS/Software-Development-Engineer-1)]
3. Thomson Reuters [[Software Engineer](https://careers.thomson

In [31]:
from twilio.rest import Client

account_sid = twilio_account_sid
auth_token = twilio_auth_token
client = Client(account_sid, auth_token)


message_body = ""
for classification, result in results.items():
    if(classification == "Job Rejection"):
      message_body += f"**{classification}**\n{result}\n\n"

message = client.messages.create(
  from_=twilio_from,
  body=message_body,
  to=twilio_to
)

print(message.sid)

SM5f607161d02b040f4da3d39e77f7a9ab


In [ ]:
#TODO: Delete all the email files from emails folder after the job is done
#TODO: Integrate Twilio
#TODO: Filter out the not job specific emails and do not send them to the database
#TODO: Fix classification for the emails/ add priority to the classifications
#TODO: Segregate the summarize and classification prompts and functions
#TODO: Run every 24 hours using a cron job/kafka
#TODO: Update tags based on existing companies and roles
#TODO: Optimise the code
#TODO: Try to add the data to a spreadsheet, and then use the spreadsheet as the source for the data
#TODO: Use twilio to send whatsapp notifications based on the classification